In [2]:
%matplotlib inline
%load_ext autoreload

In [3]:
import os
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn

from sklearn.cluster import KMeans

In [23]:
df = pd.read_parquet("s3://iefp-unemployment/intermediate/clean/interventions.parquet")
df.head()

,ute_id,tipo_utente,centro,tipo_movimento,data_intervencao,codigo_intervencao,cnp_pretendida,area_intervencao,data_resultado,resultado_intervencao,...,f_dcurso,f_horas,f_cmod_form,f_ccurso,f_dmod_form,f_carea_form,f_darea_form,f_vagas,f_vagas_ocupadas,cest_superior
0,6905559,A,555,35,2019-07-01,101,NaN,NaN,NaN,20,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
1,6905559,A,555,35,2019-07-02,103,NaN,NaN,NaN,20,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
3,6905559,A,555,35,2019-07-02,212,NaN,NaN,NaN,1,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
6,6905559,A,555,35,2019-07-01,7003,NaN,99.0,NaN,1,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
7,88081,A,341,35,2019-07-01,226,NaN,NaN,NaN,1,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN


## Join EFA

In [9]:
#efa_interv = ["742","767","768","769","770","771","772","779","784","785"]

In [13]:
efa_interv = [742,767,768,769,770,771,772,779,784,785]

In [10]:
print(efa_interv)

['742', '767', '768', '769', '770', '771', '772', '779', '784', '785']


In [16]:
df.codigo_intervencao.isin([742,767,768,769,770,771,772,779,784,785]).value_counts()

False    5826188
True       55051
Name: codigo_intervencao, dtype: int64

In [18]:
df.loc[(df.codigo_intervencao.isin([742,767,768,769,770,771,772,779,784,785]))] = 1000

In [24]:
def split_efa(df):
    df.loc[(df.codigo_intervencao.isin([742,767,768,769,770,771,772,779,784,785]))] = 1000
    return df

In [25]:
df = split_efa(df)

In [26]:
df.codigo_intervencao.value_counts()

7003    1441259
101     1322453
7001     844744
103      747423
102      231138
225      174930
399      174871
777      160919
7013     127083
7016     107793
7002      84772
398       57194
1000      55051
9902      53417
506       45961
427       39975
208       33364
507       22876
416       18234
230       13623
7005      13519
226       13429
394       11067
395        9154
700        8783
611        8006
7017       6554
794        5816
791        5322
796        5299
         ...   
421           4
213           3
721           3
7009          3
764           3
329           3
422           3
415           3
607           3
233           3
717           2
408           2
750           2
430           2
706           2
713           2
220           2
216           1
508           1
714           1
7104          1
738           1
305           1
752           1
417           1
708           1
766           1
407           1
325           1
707           1
Name: codigo_intervencao

In [9]:
df = pd.read_parquet("s3://iefp-unemployment/intermediate/clean/interventions.parquet")
df.head()

,ute_id,tipo_utente,centro,tipo_movimento,data_intervencao,codigo_intervencao,cnp_pretendida,area_intervencao,data_resultado,resultado_intervencao,...,f_dcurso,f_horas,f_cmod_form,f_ccurso,f_dmod_form,f_carea_form,f_darea_form,f_vagas,f_vagas_ocupadas,cest_superior
0,88081,A,341,35,2019-07-01,226,NaN,NaN,NaN,1,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
1,6905559,A,555,35,2019-07-02,103,NaN,NaN,NaN,20,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
4,6905559,A,555,35,2019-07-02,212,NaN,NaN,NaN,1,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
7,6905559,A,555,35,2019-07-01,7003,NaN,99.0,NaN,1,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
8,6905559,A,555,35,2019-07-01,101,NaN,NaN,NaN,20,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN


In [16]:
df.loc[(df.codigo_intervencao == 777),"f_cmod_form"].drop_duplicates()

200       805.0
350       803.0
6410        NaN
17952     802.0
29407     801.0
56698     796.0
106078    797.0
167373    777.0
379824    804.0
399084    785.0
Name: f_cmod_form, dtype: float64

## Break down 

In [74]:
df = pd.read_parquet("s3://iefp-unemployment/intermediate/clean/interventions.parquet")
df.loc[
    (df.codigo_intervencao == 777) & (df.f_cmod_form.notna()), "codigo_intervencao"
] = (
    "777_"
    + (
        df.loc[
            (df.codigo_intervencao == 777) & (df.f_cmod_form.notna()), "f_cmod_form"
        ].astype(int).astype(str)
    )
)

In [75]:
df.head()

,ute_id,tipo_utente,centro,tipo_movimento,data_intervencao,codigo_intervencao,cnp_pretendida,area_intervencao,data_resultado,resultado_intervencao,...,f_dcurso,f_horas,f_cmod_form,f_ccurso,f_dmod_form,f_carea_form,f_darea_form,f_vagas,f_vagas_ocupadas,cest_superior
0,6905559,A,555,35,2019-07-01,101,NaN,NaN,NaN,20,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
1,6905559,A,555,35,2019-07-02,103,NaN,NaN,NaN,20,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
3,6905559,A,555,35,2019-07-02,212,NaN,NaN,NaN,1,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
6,6905559,A,555,35,2019-07-01,7003,NaN,99.0,NaN,1,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
7,88081,A,341,35,2019-07-01,226,NaN,NaN,NaN,1,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN


In [70]:

def add_training(df):
        """Function breaks down training types in interventons table
        into separate interventions"""

        df.loc[
            (df.codigo_intervencao == 777) &
            (df.f_cmod_form.notna()), "codigo_intervencao"
        ] = (
            "777_"
            + (
                df.loc[
                    (df.codigo_intervencao == 777) &
                    (df.f_cmod_form.notna()), "f_cmod_form"
                ]
                .astype(int)
                .astype(str)
            )
        )
        return df

In [81]:
trial = pd.read_parquet("s3://iefp-unemployment/intermediate/transform/intermediate_trial.parquet")
trial.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['user_id', 'journey_count', 'register_date', 'exit_date',
       'register_movement', 'exit_movement', 'register_category',
       'exit_category', 'register_reason', 'exit_reason', 'ute_id',
       'data_movimento', 'd_birth_date', 'd_gender', 'd_age', 'd_disabled',
       'd_dependents', 'd_professional_training', 'd_profession_card',
       'd_school_qualification', 'd_college_qualification', 'd_civil_status',
       'd_registration_type', 'd_nationality', 'd_parish', 'd_center',
       'd_subsidy', 'd_rsi', 'd_previous_cpp', 'd_previous_job_experience',
       'd_desired_cpp', 'd_desired_location', 'd_desired_work_time',
       'd_desired_contract', 'intervention_date_1000', 'intervention_date_207',
       'intervention_date_208', 'intervention_date_218',
       'intervention_date_219', 'intervention_date_225',
       'intervention_date_226', 'intervention_date_235',
       'intervention_date_319', 'intervention_date_321',
       'interv

In [79]:
trial['intervention_date_777777'].value_counts() #8
trial['intervention_date_777785'].value_counts() #5 *
trial['intervention_date_777796'].value_counts() #+
trial['intervention_date_777797'].value_counts() #++
trial['intervention_date_777801'].value_counts() #2
trial['intervention_date_777802'].value_counts() #4 *
trial['intervention_date_777803'].value_counts() #++ *
trial['intervention_date_777805'].value_counts() #++ *

2018-12-10    115
2018-11-26    103
2018-12-03     84
2018-11-05     77
2019-03-11     61
2018-11-19     56
2019-02-18     54
2018-10-15     51
2018-12-11     49
2018-11-12     49
2019-02-11     45
2019-02-04     45
2018-12-17     45
2018-12-05     42
2018-11-22     42
2019-02-25     41
2018-12-18     41
2018-11-15     41
2019-01-28     40
2018-12-04     39
2018-10-09     37
2018-10-25     37
2019-03-25     36
2018-10-10     36
2018-12-27     35
2019-01-23     34
2018-10-24     33
2018-11-21     33
2018-09-10     32
2018-11-20     30
             ... 
2019-05-13      2
2018-07-02      2
2018-12-09      2
2018-09-25      2
2018-09-28      2
2018-07-23      2
2018-06-25      2
2019-02-22      1
2018-06-21      1
2019-04-11      1
2019-01-16      1
2018-06-14      1
2019-01-08      1
2019-05-04      1
2018-05-24      1
2018-12-15      1
2018-04-17      1
2018-11-02      1
2019-03-19      1
2018-07-12      1
2019-04-12      1
2018-02-15      1
2019-05-02      1
2018-09-14      1
2018-04-16

## group training

In [65]:
df = pd.read_parquet("s3://iefp-unemployment/intermediate/clean/interventions.parquet")

df.dtypes

ute_id                            int32
tipo_utente                      object
centro                            int16
tipo_movimento                     int8
data_intervencao         datetime64[ns]
codigo_intervencao                int16
cnp_pretendida                  float64
area_intervencao                float64
data_resultado                  float64
resultado_intervencao              int8
qualificacao                    float64
areas_curso                      object
tempo_pratica_ucnp              float64
tipo                             object
tipo_encaminhamento              object
ccentro                         float64
f_dcurso                         object
f_horas                         float64
f_cmod_form                     float64
f_ccurso                         object
f_dmod_form                      object
f_carea_form                    float64
f_darea_form                     object
f_vagas                         float64
f_vagas_ocupadas                float64


In [67]:
def group_training(df):
    df.loc[(df.codigo_intervencao.isin([777777,777785,777802,777803,777805,777801])),'codigo_intervencao'] = 777777
    return df

In [76]:
df = add_training(df)
df.codigo_intervencao.drop_duplicates().tolist()

[101,
 103,
 212,
 7003,
 226,
 7001,
 416,
 772,
 '777_803',
 208,
 '777_805',
 7013,
 9902,
 399,
 102,
 7002,
 398,
 794,
 225,
 785,
 507,
 700,
 506,
 7016,
 770,
 735,
 613,
 395,
 611,
 784,
 609,
 235,
 733,
 7007,
 394,
 742,
 9904,
 610,
 769,
 230,
 777,
 768,
 767,
 603,
 302,
 7005,
 7018,
 207,
 501,
 232,
 214,
 299,
 328,
 729,
 319,
 403,
 7017,
 203,
 202,
 201,
 9999,
 '777_802',
 217,
 601,
 791,
 502,
 503,
 718,
 728,
 722,
 795,
 796,
 797,
 404,
 428,
 778,
 724,
 704,
 414,
 7004,
 771,
 323,
 304,
 '777_801',
 705,
 745,
 '777_796',
 504,
 321,
 702,
 9901,
 427,
 '777_797',
 320,
 402,
 215,
 9907,
 615,
 405,
 418,
 301,
 '777_777',
 716,
 411,
 231,
 710,
 9903,
 7019,
 205,
 701,
 206,
 614,
 505,
 218,
 779,
 233,
 744,
 '777_804',
 409,
 739,
 703,
 605,
 401,
 9906,
 713,
 204,
 764,
 715,
 720,
 327,
 329,
 430,
 7008,
 234,
 417,
 711,
 709,
 746,
 749,
 612,
 714,
 219,
 606,
 396,
 750,
 210,
 707,
 736,
 719,
 743,
 775,
 706,
 602,
 607,
 325,
 '7

In [75]:

df = group_training(df)
df.codigo_intervencao.drop_duplicates().tolist()

[101,
 103,
 212,
 7003,
 226,
 7001,
 416,
 772,
 '777_803',
 208,
 '777_805',
 7013,
 9902,
 399,
 102,
 7002,
 398,
 794,
 225,
 785,
 507,
 700,
 506,
 7016,
 770,
 735,
 613,
 395,
 611,
 784,
 609,
 235,
 733,
 7007,
 394,
 742,
 9904,
 610,
 769,
 230,
 777,
 768,
 767,
 603,
 302,
 7005,
 7018,
 207,
 501,
 232,
 214,
 299,
 328,
 729,
 319,
 403,
 7017,
 203,
 202,
 201,
 9999,
 '777_802',
 217,
 601,
 791,
 502,
 503,
 718,
 728,
 722,
 795,
 796,
 797,
 404,
 428,
 778,
 724,
 704,
 414,
 7004,
 771,
 323,
 304,
 '777_801',
 705,
 745,
 '777_796',
 504,
 321,
 702,
 9901,
 427,
 '777_797',
 320,
 402,
 215,
 9907,
 615,
 405,
 418,
 301,
 '777_777',
 716,
 411,
 231,
 710,
 9903,
 7019,
 205,
 701,
 206,
 614,
 505,
 218,
 779,
 233,
 744,
 '777_804',
 409,
 739,
 703,
 605,
 401,
 9906,
 713,
 204,
 764,
 715,
 720,
 327,
 329,
 430,
 7008,
 234,
 417,
 711,
 709,
 746,
 749,
 612,
 714,
 219,
 606,
 396,
 750,
 210,
 707,
 736,
 719,
 743,
 775,
 706,
 602,
 607,
 325,
 '7

In [44]:
trial.head()

,user_id,journey_count,register_date,exit_date,register_movement,exit_movement,register_category,exit_category,register_reason,exit_reason,...,intervention_date_791,intervention_date_794,intervention_date_795,intervention_date_796,intervention_date_797,intervention_date_9902,intervention_date_9903,intervention_date_9904,intervention_date_9906,intervention_date_9907
0,108,1,2016-11-10 17:00:31,2018-07-31 20:06:21,43.0,43.0,2.0,3.0,3.0,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,304,1,2017-01-17 09:51:43,2017-11-21 16:46:03,11.0,31.0,2.0,2.0,7.0,11.0,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,724,4,2018-04-18 10:58:08,2018-12-19 11:02:11,11.0,31.0,2.0,2.0,15.0,44.0,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,757,1,2017-01-26 13:10:33,2017-07-19 12:35:24,11.0,31.0,2.0,2.0,12.0,20.0,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,899,1,2016-10-04 13:13:39,2017-04-24 09:58:04,11.0,31.0,2.0,2.0,7.0,19.0,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [41]:
trial.codigo_intervencao.value_counts()

AttributeError: 'DataFrame' object has no attribute 'codigo_intervencao'

In [79]:

def add_training(df):
    for i in df.loc[df.codigo_intervencao == 777, "codigo_intervencao"]:
        df.replace(i,"777_"+"{}".format(df["f_cmod_form"].astype(str)))
        return df



In [80]:
df_n = add_training(df_f)
df_n.head()


,ute_id,tipo_utente,centro,tipo_movimento,data_intervencao,codigo_intervencao,cnp_pretendida,area_intervencao,data_resultado,resultado_intervencao,...,f_dcurso,f_horas,f_cmod_form,f_ccurso,f_dmod_form,f_carea_form,f_darea_form,f_vagas,f_vagas_ocupadas,cest_superior
0,6905559,A,555,35,2019-07-01,101,NaN,NaN,NaN,20,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
1,6905559,A,555,35,2019-07-02,103,NaN,NaN,NaN,20,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
3,6905559,A,555,35,2019-07-02,212,NaN,NaN,NaN,1,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
6,6905559,A,555,35,2019-07-01,7003,NaN,99.0,NaN,1,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
7,88081,A,341,35,2019-07-01,226,NaN,NaN,NaN,1,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN


In [81]:
df = pd.read_csv("s3://iefp-unemployment/SIGAE/MODALIDADES_FORMACAO/modalidades_formacao.csv", encoding = 'latin_1')
df.head()

,CMODALIDADE_QCA,DMODALIDADE_QCA,HMODALIDADE_QCA,CMODALIDADE_QCA_AGR,DMODALIDADE_QCA_AGR,SUPORTA_UFCD,DMODALIDADE_QCA_AGR2,DMODALIDADE_QCA_AGR_INTERNET
0,1155,CURSOS EFA TIPO B2+B3,A,771,FORMAÌO CONTêNUA - EFA TIPO B2+B3,S,EFA,Cursos de educao formao de adultos (equiva...
1,1150,Cursos EFA Tipo B2+B3 (escolar),A,771,FORMAÌO CONTêNUA - EFA TIPO B2+B3,S,EFA,Cursos de educao formao de adultos (equiva...
2,1157,CURSOS EFA - BASICO - PROFISSIONAL,A,789,EFA - BçSICO PROFISSIONAL,S,EFA,Cursos de educao e formao de adultos (nve...
3,1231,FORMAÌO INICIAL - PFE'S,H,734,"FORM. CONTêNUA - RECICLAGEM, ACTUALIZAÌO E AP...",N,FORMAÌO CONTINUA,"FORM. CONTêNUA - RECICLAGEM, ACTUALIZAÌO E AP..."
4,2101,FORMAÌO CONTêNUA - SABER+,H,734,"FORM. CONTêNUA - RECICLAGEM, ACTUALIZAÌO E AP...",N,FORMAÌO CONTINUA,"FORM. CONTêNUA - RECICLAGEM, ACTUALIZAÌO E AP..."


In [82]:
yam = df[["CMODALIDADE_QCA_AGR","DMODALIDADE_QCA_AGR2"]]
yam = yam.drop_duplicates()
yam

,CMODALIDADE_QCA_AGR,DMODALIDADE_QCA_AGR2
0,771,EFA
2,789,EFA
3,734,FORMAÌO CONTINUA
10,784,EFA
12,785,EFA
13,772,EFA
14,773,EFA
15,774,EFA
16,777,EFA
17,777,FORMAÌO CONTINUA


In [83]:
df.CMODALIDADE_QCA_AGR.drop_duplicates().shape

(53,)

In [84]:
#dmodalidade_qca_agr2

In [85]:
df_f.f_cmod_form[df_f["f_cmod_form"].isin(yam.CMODALIDADE_QCA_AGR)].drop_duplicates().shape

(33,)

In [86]:
print(yam.to_csv(sep=':', index=False, header=False))

771:EFA
789:EFA
734:FORMAÌO CONTINUA
784:EFA
785:EFA
772:EFA
773:EFA
774:EFA
777:EFA
777:FORMAÌO CONTINUA
778:FORMAÌO CONTINUA
779:EFA
736:FORMAÌO ACTIVOS QUALIFICADOS
731:EDUCAÌO FORMAÌO JOVENS
765:FORMAÌO Ë MEDIDA
780:EFA
781:EFA
782:EFA
783:EFA
751:EDUCAÌO FORMAÌO JOVENS
752:EDUCAÌO FORMAÌO JOVENS
753:EDUCAÌO FORMAÌO JOVENS
754:EDUCAÌO FORMAÌO JOVENS
755:EDUCAÌO FORMAÌO JOVENS
759:EDUCAÌO FORMAÌO JOVENS
756:EDUCAÌO FORMAÌO JOVENS
757:EDUCAÌO FORMAÌO JOVENS
758:EDUCAÌO FORMAÌO JOVENS
760:EDUCAÌO FORMAÌO JOVENS
728:FORMAÌO PEDAGîGICA
733:ESPECIALIZAÌO TECNOLîGICA
732:QUALIFICAÌO INICIAL
727:FORMAÌO CONTINUA
729:FORMAÌO PEDAGîGICA
737:FORMAÌO CONTINUA
797:FORMAÌO CONTINUA
795:FORMAÌO CONTINUA
796:FORMAÌO CONTINUA
735:FORMAÌO CONTINUA
791:FORMAÌO EM COMPETæNCIAS BçSICAS
801:FORMAÌO CONTINUA
805:FORMAÌO CONTINUA
802:FORMAÌO CONTINUA
803:FORMAÌO CONTINUA
804:ESPECIALIZAÌO TECNOLîGICA
804:EFA
738:FORMAÌO CONTINUA
700:APRENDIZAGEM
730:EFA
763:FORM

In [105]:
trial = pd.read_parquet("s3://iefp-unemployment/intermediate/transform/intermediate_trial.parquet")
trial.head()
trial.columns.values

array(['ute_id', 'tipo_utente', 'centro', 'tipo_movimento',
       'data_intervencao', 'codigo_intervencao', 'cnp_pretendida',
       'area_intervencao', 'data_resultado', 'resultado_intervencao',
       'qualificacao', 'areas_curso', 'tempo_pratica_ucnp', 'tipo',
       'tipo_encaminhamento', 'ccentro', 'f_dcurso', 'f_horas',
       'f_cmod_form', 'f_ccurso', 'f_dmod_form', 'f_carea_form',
       'f_darea_form', 'f_vagas', 'f_vagas_ocupadas', 'cest_superior'],
      dtype=object)

In [107]:
trial.codigo_intervencao.drop_duplicates()

0          101
1          103
3          212
6         7003
7          226
9         7001
37         416
38         772
57         777
70         208
106       7013
131       9902
192        399
193        102
209       7002
214        398
228        794
404        225
438        785
468        507
548        700
605        506
678       7016
786        770
808        735
1372       613
2046       395
2591       611
2710       784
2746       609
          ... 
87359      738
159390     730
160040     213
195776     429
260010    7021
345307     305
353329     787
509309     324
274107     407
324638     236
390255     717
461804     766
476457     747
370348     222
7356       721
22835     7102
79870      220
96443      604
152886     216
563675     753
193065     752
13412      421
192081     508
535293     708
296983    7010
502873     789
516468     422
231984     415
269213    7109
531000    7104
Name: codigo_intervencao, Length: 183, dtype: int16

In [17]:
mod = pd.read_parquet("s3://iefp-unemployment/modelling/modelling.parquet")
mod.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['user_id', 'journey_count', 'register_date', 'exit_date',
       'register_movement', 'exit_movement', 'register_category',
       'exit_category', 'register_reason', 'exit_reason', 'ute_id',
       'data_movimento', 'd_birth_date', 'd_gender', 'd_age', 'd_disabled',
       'd_dependents', 'd_professional_training', 'd_profession_card',
       'd_school_qualification', 'd_college_qualification', 'd_civil_status',
       'd_registration_type', 'd_nationality', 'd_parish', 'd_center',
       'd_subsidy', 'd_rsi', 'd_previous_job_experience', 'd_desired_location',
       'd_desired_work_time', 'd_desired_contract', 'i_efa',
       'i_rvcc_other_entities', 'i_job_search_techn', 'i_professional_rvcc',
       'i_rvcc_integrated', 'i_train_active_life',
       'i_tutoring_in_individual_job_search', 'i_job_search_assistance',
       'i_pae_cpe_subsidy', 'i_paper_subsidy_ace', 'i_paecpe_ace',
       'i_business_and_employment_creation_help',
       '

In [8]:
mod["i_training_feg_professional"].value_counts()

2018-12-10    117
2018-11-26    115
2018-12-03     91
2018-11-05     78
2019-03-11     62
2018-11-19     61
2018-10-15     56
2019-02-18     55
2018-11-12     53
2018-12-11     50
2018-12-17     47
2019-02-11     47
2019-02-04     47
2019-02-25     44
2018-12-05     44
2018-11-22     44
2018-11-15     42
2018-12-18     42
2019-01-28     42
2018-12-04     41
2018-10-09     39
2019-03-25     38
2018-10-25     37
2018-10-10     36
2018-12-27     35
2018-11-21     35
2018-09-10     34
2018-10-24     34
2019-01-23     33
2018-09-19     31
             ... 
2018-12-15      1
2019-01-08      1
2018-07-12      1
2017-12-05      1
2019-03-19      1
2016-09-20      1
2019-05-23      1
2019-05-04      1
2017-05-11      1
2017-05-16      1
2018-07-20      1
2018-04-16      1
2018-07-10      1
2016-10-10      1
2018-09-14      1
2016-10-17      1
2018-12-28      1
2016-05-27      1
2019-04-11      1
2016-10-24      1
2018-04-17      1
2019-05-08      1
2015-09-18      1
2017-07-03      1
2018-11-02